## The Impact of Tuition and Financial Aid on Student Retention and Post-Graduation Outcomes
#### Ava Lasater
#### INFO 523 Data Mining

### Import Statements

In [2]:
import pandas as pd

### Data Loading
*Since the data file was so large, I had to split the data using excel to break it down into files that could be uploaded. Since the files are still large, I will be loading then removing columns before joining the data frames back together.*

In [3]:
# using pandas to upload the data chunks

AK_DE = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardAK_DE.csv')
FL_KS = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardFL_KS.csv')
KY_MN = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardKY_MN.csv')
MO_NJ = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardMO_NJ.csv')
NM_OR = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardNM_OR.csv')
PA_TX = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardPA_TX.csv')
UT_WY = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardUT_WY.csv')

/tmp/ipykernel_2042/2842618077.py:3: DtypeWarning: Columns (1727,1957,1958,1959,1960,1969,1970,1971,1972,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530) have mixed types. Specify dtype option on import or set low_memory=False.
  AK_DE = pd.read_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecardAK_DE.csv')
/tmp/ipykernel_2042/2842618077.py:4: DtypeWarning: Columns (6,9,1608,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,25

In [5]:
# performing some basic data cleaning
# I will be deleting columns that have Pell Grant data because I want to focus on other aspects of college data
# will be doing this for each chunk of data so that it is less intesive to join the data later

print('Size of first data chunk before getting rid of Pell Grant columns:', AK_DE.shape)
AK_DE_NP = AK_DE.drop(AK_DE.filter(regex='PELL', axis=1).columns, axis=1)
print('Size of first data chunk after getting rid of Pell Grant columns:', AK_DE_NP.shape)
FL_KS_NP = FL_KS.drop(FL_KS.filter(regex='PELL', axis=1).columns, axis=1)
KY_MN_NP = KY_MN.drop(KY_MN.filter(regex='PELL', axis=1).columns, axis=1)
MO_NJ_NP = MO_NJ.drop(MO_NJ.filter(regex='PELL', axis=1).columns, axis=1)
NM_OR_NP = NM_OR.drop(NM_OR.filter(regex='PELL', axis=1).columns, axis=1)
PA_TX_NP = PA_TX.drop(PA_TX.filter(regex='PELL', axis=1).columns, axis=1)
UT_WY_NP = UT_WY.drop(UT_WY.filter(regex='PELL', axis=1).columns, axis=1)

# now that some of the columns have been removed, I am going to join all of the data chunks into one dataframe
college_data = pd.concat([AK_DE_NP, FL_KS_NP, KY_MN_NP, MO_NJ_NP, NM_OR_NP, PA_TX_NP, UT_WY_NP], ignore_index=True)
print('Size of combined data:', college_data.shape)

# it looks like we gained some rows somehow. lets check for duplicates and remove them
duplicates = college_data.duplicated().sum()
print('Number of duplicate rows in combined data:', duplicates)
college_data_ND = college_data.drop_duplicates()
print('Size of combined data after removing duplicates:', college_data_ND.shape)

# now lets do a bit more work on getting only columns that might be of interest to us
'''
from looking at the College Scorecard data dictionary, I think we can remove the following columns:
- UNITID: unique identifier for each institution, not needed for analysis
- OPEID: another identifier, not needed for analysis
- CITY, ZIP, REGION: location data, we can keep state abbreviation for regional analysis but not needed for analysis
- ADDR: address, not needed for analysis
- INSTURL: url for institution, not needed for analysis
- NPCURL: url for net price calculator, not needed for analysis
- ST_FIPS: standard code for states, not needed for analysis
- LONGITUDE, LATITUDE: geographic coordinates, not needed for analysis
- LOCALE & LOCALE2: locale codes, not needed for high-level analysis
- ACCREDAGENCY: accreditation agency, not needed for high-level analysis
- NUMBRANCH: number of branches, not needed for high-level analysis
- RELAFFIL: religous affiliation for the university, not needed for high-level analysis
- CIP fields: detailed program classification codes and if they are offered, not needed for this analysis
- CARNEGIE: Carnegie classification, while this could be interesting, it is out of scope for this project
- SCH_DEG: highest degree offered, while this could be interesting, it is out of scope for this project
- MAIN: main campus indicator, not needed for this analysis
- HCM2: Schools that are on Hieghtened Cash Monitoring, not needed for this analysis

In terms of other columns which may be interesting, but are out of scope for this project, we will remove:
- all columns concerning the faculty demographics (IRPS columns)   
- AVGFACSAL: average faculty salary, not needed for this analysis
- PFTFAC: number of full-time faculty, not needed for this analysis
- similarly to the PELL columns, columns concerning Title IV funding will also be removed as it is out of scope for this project (NUM4 columns)
- DEATH columns: data on student deaths, not needed for this analysis
- 1STGEN columns: data on first-generation students, not needed for this analysis, but could be interesting for future analysis
- TRANS columns: data on transfer students, while this could be see as a factor of student success and outcomes, it is out of scope for this project and the retention
    and graduation columns should suffice for our analysis
- SAT columns: while SAT scores could be an interesting factor to consider, they are not available for all institutions and may introduce bias into the analysis, especially considering
    that many colleges are moving towards test-optional admissions policies. Therefore, we will exclude SAT columns from our analysis to maintain consistency across institutions.
- ACT columns: similar to SAT columns, ACT scores may not be available for all institutions and could introduce bias. We will exclude ACT columns from our analysis as well.
- WITHDRAW columns: data on student withdrawals, while this could be interesting, it is out of scope for this project and we have retention and graduation columns to analyze 
    student success.
'''
columns_to_drop = ['UNITID', 'OPEID', 'OPEID6', 'CITY', 'ZIP', 'ADDR', 'ST_FIPS', 'LONGITUDE', 'LATITUDE', 'REGION', 'LOCALE', 'LOCALE2', 'INSTURL', 'NPCURL', 'AVGFACSAL', 'PFTFAC', 
                   'ACCREDAGENCY', 'NUMBRANCH', 'RELAFFIL', 'MAIN', 'HCM2', 'SCH_DEG']
college_data_ND = college_data_ND.drop(columns=columns_to_drop, axis=1)
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='CIP', axis=1).columns, axis=1)
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='CARNEGIE', axis=1).columns, axis=1)
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='IRPS', axis=1).columns, axis=1)  
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='NUM4', axis=1).columns, axis=1) 
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='DEATH', axis=1).columns, axis=1) 
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='1STGEN', axis=1).columns, axis=1)
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='TRANS', axis=1).columns, axis=1)
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='SAT', axis=1).columns, axis=1)
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='ACT', axis=1).columns, axis=1)
college_data_ND = college_data_ND.drop(college_data_ND.filter(regex='WITHDRAW', axis=1).columns, axis=1)

print('Size of combined data after dropping unneeded columns:', college_data_ND.shape) 

# finally, for the last part of the inital data cleaning to make this dataset more manageable, I will save this datafile as a new csv and this is the dataset that I will refere to in the
    # data README file and use for the rest of the analysis
college_data_ND.to_csv('/workspaces/final-project-ava-lasater/data/CollegeScorecard_Cleaned.csv', index=False)

Size of first data chunk before getting rid of Pell Grant columns: (1167, 3306)
Size of first data chunk after getting rid of Pell Grant columns: (1167, 2824)
Size of combined data: (6903, 2824)
Number of duplicate rows in combined data: 473
Size of combined data after removing duplicates: (6430, 2824)
Size of combined data after dropping unneeded columns: (6430, 1911)
